# Yield Model Calculator for W2W hybrid bonding
#### Author: Zhichao Chen
#### Date: Sep 16, 2024
#### Description: V2 (overlay errors, particulate defects, Cu expansion, surface roughness)

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from assembly_yield_simulator import Assembly_Yield_Simulator
import sys

In [3]:
# WAFER PARAMETERS
NUM_WAFERS = 1
# PER WAFER PARAMETERS
PITCH = 6  # pitch (um)
DIE_W = 1e+4  # die width (um)
DIE_L = 1e+4  # die length (um)
WAF_R = 150e+3  # wafer radius (um)
PAD_TOP_R = 1.0  # top Cu pad radius (um)
PAD_BOT_R = 1.5  # bottom Cu pad radius (um)
PAD_ARR_ROW = int(np.floor(float(DIE_L / PITCH)))  # number of pads in a row of pad array
PAD_ARR_COL = int(np.floor(float(DIE_W / PITCH)))  # number of pads in a column of pad array
VOID_SHAPE = "circle"  # void shape: 'circle' or 'square'
dice_width = 10000 # dice width (um)

# Overlay Model Parameters
RANDOM_MISALIGNMENT_MEAN = 5e-3
RANDOM_MISALIGNMENT_STD = 1e-2
# Assume the systematic translation is normally distributed
SYSTEM_TRANSLATION_X_MEAN = 5e-3  # systematic translation mean (um) - x direction
SYSTEM_TRANSLATION_X_STD = 1e-2  # systematic translation standard deviation (um)  - x direction
SYSTEM_TRANSLATION_Y_MEAN = 5e-3  # systematic translation mean (um) - y direction
SYSTEM_TRANSLATION_Y_STD = 1e-2  # systematic translation standard deviation (um)  - y direction
# Assume the systematic rotation is normally distributed
SYSTEM_ROTATION_MEAN = 1e-7  # systematic rotation mean (rad)
SYSTEM_ROTATION_STD = 0.5e-7  # systematic rotation standard deviation (rad)
# Assume the bow difference is normally distributed
BOW_DIFFERENCE_MEAN = 10  # bow difference mean (um)
BOW_DIFFERENCE_STD = 3  # bow difference standard deviation (um)
# systematic magnification model: e = k_mag * bow_diff
k_mag = 0.09
M_0 = 0
SYSTEM_MAGNIFICATION_MEAN = (k_mag * BOW_DIFFERENCE_MEAN + M_0) / 1e6  # systematic magnification mean (ppm)
SYSTEM_MAGNIFICATION_STD = (k_mag * BOW_DIFFERENCE_STD) ** 2 / 1e6 # systematic magnification standard deviation (ppm)

# Defect Model Parameters
D0 = 1e-9  # Number of particles of all thicknesses per unit area (um^{-1})
z = 3  # Exponential factor of the particle thickness distribution
t_0 = 1  # The smallest particle thickness (um)
L_m = WAF_R  # Beyond this distance, no voids in the void tail
k_r = 1.8e-4
k_r0 = 230
k_L = 6.2e-2
k_n = 9e-5
k_S = 2.7
cluster_para = 0.1

# Cu expansion model parameters
# Cu pad dish mean and standard deviation at room temperature
TOP_DISH_MEAN = -10.0        # Top Cu pad dish mean (nm), negative value means the dish is concave
TOP_DISH_STD = 1.0          # Top Cu pad dish standard deviation (nm)
BOT_DISH_MEAN = -10.0        # Bottom Cu pad dish mean (nm), negative value means the dish is concave
BOT_DISH_STD = 1.0          # Bottom Cu pad dish standard deviation (nm)
# Expansion coefficient of Cu pad,      ΔH = k_e * ΔT
k_et = 0.052            # Top Cu pad expansion/temp coefficient (nm/K)
k_eb = 0.052            # Bottom Cu pad expansion/temp coefficient (nm/K)
T_R = 25                # Room temperature (°C)
T_anl = 300             # Annealing temperature (°C)

# Roughness model parameters
Asperity_R = 2e-6  # Asperity curvature radius (m)
Roughness_sigma = 1.0e-9  # Surface roughness standard deviation (m)
eta_s = 40e+12  # Asperity density (m^{-2})
Roughness_constant = 0.07
Adhesion_energy = 1.2  # Adhesion energy (J/m^2)
Young_modulus = 73e+9  # Young's modulus of dielectric material (Pa)
Dielectric_thickness = 1.5e-6  # Dielectric thickness (m)
DISH_0 = 75e-9  # Reference dish (m)
k_peel = 6.66e+15  # Peel force constant (N/m^4/K)

In [12]:
# DIE PARAMETERS
NUM_DIES = 10000
# PER DIE PARAMETERS
PITCH = 6  # pitch (um)
DIE_W = 3e+4  # die width (um)
DIE_L = 3e+4  # die length (um)
WAF_R = 150e+3  # wafer radius (um)
PAD_TOP_R = 1.0  # top Cu pad radius (um)
PAD_BOT_R = 1.5  # bottom Cu pad radius (um)
PAD_ARR_ROW = int(np.floor(float(DIE_L / PITCH)))  # number of pads in a row of pad array
PAD_ARR_COL = int(np.floor(float(DIE_W / PITCH)))  # number of pads in a column of pad array
VOID_SHAPE = "circle"  # void shape: 'circle' or 'square'
dice_width = 0  # dice width (um)


simulation_times = 1

num_points = 10
# Explore the impact of the pitch
die_size_list = np.linspace(8e+4, 8e+4, num_points)
pitch_list = np.linspace(10, 10, num_points)
particle_density_list = np.logspace(-10, -8, num_points)
t_0_list = np.linspace(0.1, 0.1, num_points)
dish_std_list = np.linspace(1, 1, num_points)
translation_list = np.linspace(1e-2, 1e-2, num_points)
rotation_mean_list = np.logspace(-7, -7, num_points)
Roughness_sigma_list = np.linspace(1.0e-9, 1.0e-9, num_points)
PAD_BOT_R_ratio_list = np.linspace(0.5, 0.5, num_points)
PAD_TOP_R_ratio_list = np.linspace(2/3, 2/3, num_points)
bow_difference_mean_list = np.linspace(10, 10, num_points)
assembly_yield_list = []
single_config_yield_list_array = np.zeros([len(particle_density_list), simulation_times * NUM_DIES])
for i in range(len(particle_density_list)):
    DIE_W = die_size_list[i]  # die width (um)
    DIE_L = die_size_list[i]  # die length (um)
    PAD_ARR_ROW = int(np.floor(float(DIE_L / pitch_list[i])))  # number of pads in a row of pad array
    PAD_ARR_COL = int(np.floor(float(DIE_W / pitch_list[i])))  # number of pads in a column of pad array
    PAD_BOT_R = pitch_list[i] / 2 * PAD_BOT_R_ratio_list[i]  # bottom Cu pad radius (um)
    PAD_TOP_R = PAD_BOT_R * PAD_TOP_R_ratio_list[i]  # top Cu pad radius (um)
    print("Processing {}/{}, particle dednsity: {}, translation: {}, dish_std: {}".format(i+1, len(particle_density_list), particle_density_list[i], translation_list[i], dish_std_list[i]))
    assembly_yield, single_config_yield_list = Assembly_Yield_Simulator(
        NUM_DIES=NUM_DIES,
        PITCH=pitch_list[i],
        DIE_W=die_size_list[i],
        DIE_L=die_size_list[i],
        WAF_R=WAF_R,
        PAD_TOP_R=PAD_TOP_R,
        PAD_BOT_R=PAD_BOT_R,
        PAD_ARR_ROW=PAD_ARR_ROW,
        PAD_ARR_COL=PAD_ARR_COL,
        VOID_SHAPE=VOID_SHAPE,
        dice_width=dice_width,
        RANDOM_MISALIGNMENT_MEAN=RANDOM_MISALIGNMENT_MEAN,
        RANDOM_MISALIGNMENT_STD=RANDOM_MISALIGNMENT_STD,
        SYSTEM_TRANSLATION_X_MEAN=translation_list[i],
        SYSTEM_TRANSLATION_X_STD=SYSTEM_TRANSLATION_X_STD,
        SYSTEM_TRANSLATION_Y_MEAN=translation_list[i],
        SYSTEM_TRANSLATION_Y_STD=SYSTEM_TRANSLATION_Y_STD,
        SYSTEM_ROTATION_MEAN=SYSTEM_ROTATION_MEAN,
        SYSTEM_ROTATION_STD=SYSTEM_ROTATION_STD,
        BOW_DIFFERENCE_MEAN=bow_difference_mean_list[i],
        BOW_DIFFERENCE_STD=BOW_DIFFERENCE_STD,
        k_mag=k_mag,
        M_0=M_0,
        D0=particle_density_list[i],
        z=z,
        t_0=t_0_list[i],
        k_r=k_r,
        k_r0=k_r0,
        k_L=k_L,
        k_n=k_n,
        k_S=k_S,
        TOP_DISH_MEAN=TOP_DISH_MEAN,
        TOP_DISH_STD=dish_std_list[i],
        BOT_DISH_MEAN=BOT_DISH_MEAN,
        BOT_DISH_STD=dish_std_list[i],
        k_et=k_et,
        k_eb=k_eb,
        T_R=T_R,
        T_anl=T_anl,
        Asperity_R=Asperity_R,
        Roughness_sigma=Roughness_sigma_list[i],
        eta_s=eta_s,
        Roughness_constant=Roughness_constant,
        Adhesion_energy=Adhesion_energy,
        Young_modulus=Young_modulus,
        Dielectric_thickness = Dielectric_thickness,
        DISH_0 = DISH_0,
        k_peel = k_peel,
        simulation_times=simulation_times,      
        approximate_set=10                                                     
    )
                                                                        
    assembly_yield_list.append(assembly_yield)
    single_config_yield_list_array[i] = np.array(single_config_yield_list).flatten()
    if i % 10 == 0 or i == len(particle_density_list) - 1:
        print("The assembly yield list is: ", assembly_yield_list)
        # # Save the data
        np.save("correlation/D2W_simulation_assembly_yield_list_300_square.npy", assembly_yield_list)
        np.save("correlation/D2W_single_config_yield_list_array_300_square.npy", single_config_yield_list_array)

Processing 1/10, particle dednsity: 1e-10, translation: 0.01, dish_std: 1.0
The assembly yield is 53.08%.
The assembly yield list is:  [0.5308]
Processing 2/10, particle dednsity: 1.6681005372000558e-10, translation: 0.01, dish_std: 1.0
The assembly yield is 34.27%.
Processing 3/10, particle dednsity: 2.782559402207126e-10, translation: 0.01, dish_std: 1.0
The assembly yield is 16.46%.
Processing 4/10, particle dednsity: 4.6415888336127727e-10, translation: 0.01, dish_std: 1.0
The assembly yield is 4.70%.
Processing 5/10, particle dednsity: 7.742636826811277e-10, translation: 0.01, dish_std: 1.0
The assembly yield is 0.67%.
Processing 6/10, particle dednsity: 1.2915496650148826e-09, translation: 0.01, dish_std: 1.0
The assembly yield is 0.03%.
Processing 7/10, particle dednsity: 2.1544346900318866e-09, translation: 0.01, dish_std: 1.0
The assembly yield is 0.00%.
Processing 8/10, particle dednsity: 3.5938136638046257e-09, translation: 0.01, dish_std: 1.0
The assembly yield is 0.00%.
Pr